# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は NULL 罠のため回避（本問題では未使用）
* 判定は **主キー（name）= 国名**、表示は仕様どおりの列名と順序

## 1) 問題

* **要件**: 面積が 3,000,000 以上 **または** 人口が 25,000,000 以上の国を「big」とし、その **name, population, area** を取得する
* 入力テーブル: `World(name PK, continent, area INT, population INT, gdp BIGINT)`
* 出力仕様: `name, population, area`（順不同）

## 2) 最適解（単一クエリ）

> 本件はウィンドウ不要。シンプルな選択条件のみで最短解。

```sql
SELECT
  name,
  population,
  area
FROM World
WHERE area >= 3000000
   OR population >= 25000000;

Runtime 267 ms
Beats 81.63%

```

## 3) 代替解

> `OR` 条件が広範で、`area` と `population` に個別インデックスがある場合は `UNION` で **両索引を活かす** 戦略が有効なことがあります。

```sql
SELECT name, population, area
FROM World
WHERE area >= 3000000

UNION   -- 重複（両条件を満たす国）を 1 行に統合

SELECT name, population, area
FROM World
WHERE population >= 25000000;

Runtime 270 ms
Beats 77.25%

```

* 備考: `UNION ALL` ではなく `UNION` を使い、重複排除を行う。

## 4) 要点解説

* **方針**: 単純フィルタのみ。結果順は任意のため `ORDER BY` を付けずに I/O を最小化。
* **NULL**: `area` / `population` が `NULL` の行は比較結果が `UNKNOWN` となり除外される（要件的に問題なし）。
* **インデックス**: `OR` は 1 つの条件側インデックスしか使われないことがあるため、データ分布次第で `UNION` 版が有利。
* **出力列**: 仕様どおり `name, population, area` の順で投影。

## 5) 計算量（概算）

* 単純 `WHERE` 版: フルスキャンで **O(N)**、選択度が高くインデックス利用時は **O(log N + K)** 近似。
* `UNION` 版: 各枝で **O(log N + K₁)** / **O(log N + K₂)**、重複排除のマージで **O(K log K)** 程度。

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[World] --> B[WHERE area>=3000000 <br>OR population>=25000000]
  B --> C[出力 name population area]
```
